# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science 

## Homework 1: Data Collection - Web Scraping - Data Parsing

**Harvard University**<br/>
**Fall 2018**<br/>
**Instructors**: Pavlos Protopapas and Kevin Rader



<hr style='height:2px'>

In [170]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

#### Instructions
- To submit your assignment follow the instructions given in Canvas.
- The deliverables in Canvas are: <br/>
    a) This python notebook with your code and answers, plus a pdf version of it (see Canvas for details),<br/>
    b) the bibtex file you created, <br/>
    c) The CSV file you created, <br/>
    d) The JSON file you created. <br/>
    
- Exercise **responsible scraping**. Web servers can become slow or unresponsive if they receive too many requests from the same source in a short amount of time. Use a delay of 10 seconds between requests in your code. This helps not to get blocked by the target website. Run the webpage fetching part of the homework only once and do not re-run after you have saved the results in the JSON file (details below). 
- Web scraping requests can take several minutes. This is another reason why you should not wait until the last minute to do this homework.
- For this assignment, we will use Python 3.5 for grading.


# Data Collection - Web Scraping - Data Parsing 
 

In this homework, your goal is to learn how to acquire, parse, clean, and analyze data. Initially you will read the data from a file, and then later scrape them directly from a website. You will look for specific pieces of information by parsing the data, clean the data to prepare them for analysis, and finally, answer some questions.

In doing so you will get more familiar with three of the common file formats for storing and transferring data, which are:
- CSV, a text-based file format used for storing tabular data that are separated by some delimiter, usually comma or space.
- HTML/XML, the stuff the web is made of.
- JavaScript Object Notation (JSON), a text-based open standard designed for transmitting structured data over the web.

In [172]:
# import the necessary libraries
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
import re as re

## Help a professor parse their publications and extract information.

### Overview

In this part your goal is to parse the HTML page of a professor containing some of his/her publications, and answer some questions. This page is provided to you in the file `data/publist_super_clean.html`. There are 45 publications in descending order from No. 244 to No. 200.


In [173]:
# use this file provided
PUB_FILENAME = 'data/publist_super_clean.html'

<div class='exercise'><b> Question 1 [40 pts]: Parsing and Converting to bibTex and CSV using Beautiful Soup and python string manipulation </b></div>

A lot of the bibliographic and publication information is displayed in various websites in a not-so-structured HTML files. Some publishers prefer to store and transmit this information in a .bibTex file which looks roughly like this (we've simplified a few things):
```
@article { 
     author = "John Doyle"
     title = "Interaction between atoms"
     URL = "Papers/PhysRevB_81_085406_2010.pdf"
     journal = "Phys. Rev. B"
     volume = "81"
}
```
You will notice that this file format is a set of items, each of which is a set of key-value pairs. In the python world, you can think of this as a list of dictionaries.
If you think about spreadsheets (as represented by CSV files), they have the same structure. Each line is an item, and has multiple features, or keys, as represented by that line's value for the column corresponding to the key.

You are given an .html file containing a list of papers scraped from the author's website and you are to write the information into .bibTex and .CSV formats. A useful tool for parsing websites is BeautifulSoup  (http://www.crummy.com/software/BeautifulSoup/) (BS).  In this problem, will parse the  file using BS, which makes parsing HTML a lot easier.

**1.1** Write a function called `make_soup` that accepts a filename for an HTML file and returns a BS object.
    
**1.2** Write a function that reads in the BS object, parses it, converts it into a list of dictionaries: one dictionary per paper. Each of these dictionaries should have the following format (with different values for each publication):
```
{'author': 'L.A. Agapito, N. Kioussis and E. Kaxiras',
 'title': '"Electric-field control of magnetism in graphene quantum dots:\n Ab initio calculations"',
 'URL': 'Papers/PhysRevB_82_201411_2010.pdf',
 'journal': 'Phys. Rev. B',
 'volume': '82'}
```


**1.3** Convert the list of dictionaries into standard .bibTex format using python string manipulation, and write the results into a file called `publist.bib`.

**1.4** Convert the list of dictionaries into standard tabular .csv format using pandas, and write the results into a file called `publist.csv`. The csv file should have a header and no integer index.

    
#### HINT 
- Inspect the HTML code for tags that indicate information chunks such as `title` of the paper.  The `find_all` method of BeautifulSoup might be useful.
- Question 1.2 is better handled if you break the code into functions, each performing a small task such as finding the author(s) for each paper. 
- Question 1.3 is effectively tackled by first using python string formatting on a template string.
- Make sure you catch exceptions when needed. 
- Make sure you check for **missing data** and handle these cases as you see fit. 


#### Resources
- [BeautifulSoup Tutorial](https://www.dataquest.io/blog/web-scraping-tutorial-python/).
- More about the [BibTex format](http://www.bibtex.org).<BR>
    
### Answers 

In [174]:
# import the necessary libraries 
from bs4 import BeautifulSoup

<span class='sq'>**1.1 Write a function called `make_soup`  ...**</span>

In [177]:
# why doesn't this function work when the contents inside the function seem to work on their own??
def make_soup(PUB_FILENAME):
    data = open(PUB_FILENAME).read()
    soup = BeautifulSoup(data, 'html.parser')
    return(soup)

In [178]:
data = open(PUB_FILENAME).read()
soup = BeautifulSoup(data, 'html.parser')

In [179]:
# check your code - print the BS object, you should get a familiar HTML page as text
# clear/remove output before making pdf
print(soup.prettify())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"
   "http://www.w3.org/TR/html4/loose.dtd">
<title>
 Kaxiras E journal publications
</title>
<head>
 <meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
 <link href="../styles/style_pubs.css" rel="stylesheet" type="text/css"/>
 <meta content="" name="description"/>
 <meta content="Kaxiras E, Multiscale Methods, Computational Materials" name="keywords"/>
</head>
<body>
 <ol start="244">
  <li>
   <a href="Papers/2011/PhysRevB_84_125411_2011.pdf" target="paper244">
    "Approaching the intrinsic band gap in suspended high-mobility graphene nanoribbons"
   </a>
   <br/>
   Ming-Wei Lin, Cheng Ling, Luis A. Agapito, Nicholas Kioussis, Yiyang Zhang, Mark Ming-Cheng Cheng,
   <i>
    PHYSICAL REVIEW B
   </i>
   <b>
    84
   </b>
   ,  125411 (2011)
   <br/>
  </li>
 </ol>
 <ol start="243">
  <li>
   <a href="Papers/2011/PhysRevB_84_035325_2011.pdf" target="paper243">
    "Effect of symmetry breaking on the optica

Your output should look **like** this:
```
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"
   "http://www.w3.org/TR/html4/loose.dtd">

<title>Kaxiras E journal publications</title>
<head>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<link href="../styles/style_pubs.css" rel="stylesheet" type="text/css"/>
<meta content="" name="description"/>
<meta content="Kaxiras E, Multiscale Methods, Computational Materials" name="keywords"/>
</head>
<body>
<ol start="244">
<li>
<a href="Papers/2011/PhysRevB_84_125411_2011.pdf" target="paper244">
"Approaching the intrinsic band gap in suspended high-mobility graphene nanoribbons"</a>
<br/>Ming-Wei Lin, Cheng Ling, Luis A. Agapito, Nicholas Kioussis, Yiyang Zhang, Mark Ming-Cheng Cheng,
<i>PHYSICAL REVIEW B </i> <b>84</b>,  125411 (2011)
<br/>
</li>
</ol>
<ol start="243">
<li>
<a href="Papers/2011/PhysRevB_84_035325_2011.pdf" target="paper243">
"Effect of symmetry breaking on the optical absorption of semiconductor nanoparticles"</a>
<br/>JAdam Gali, Efthimios Kaxiras, Gergely T. Zimanyi, Sheng Meng,
<i>PHYSICAL REVIEW B </i> <b>84</b>,  035325 (2011)
<br/>
</li>
</ol>

...

```

In [180]:
publication = soup.find('body').get_text()
print(publication)





"Approaching the intrinsic band gap in suspended high-mobility graphene nanoribbons"
Ming-Wei Lin, Cheng Ling, Luis A. Agapito, Nicholas Kioussis, Yiyang Zhang, Mark Ming-Cheng Cheng,
PHYSICAL REVIEW B  84,  125411 (2011)






"Effect of symmetry breaking on the optical absorption of semiconductor nanoparticles"
JAdam Gali, Efthimios Kaxiras, Gergely T. Zimanyi, Sheng Meng,
PHYSICAL REVIEW B  84,  035325 (2011)






"Influence of CH2 content and network defects on the elastic properties of organosilicate glasses"
Jan M. Knaup, Han Li, Joost J. Vlassak, and Efthimios Kaxiras,
PHYSICAL REVIEW B  83,  054204 (2011)






"Direct correlation of crystal structure and optical properties in wurtzite/zinc-blende
GaAs nanowire heterostructures"
Martin Heiss, Sonia Conesa-Boj, Jun Ren, Hsiang-Han Tseng, Adam Gali,
PHYSICAL REVIEW B  83,  045303 (2011)






"Endothelial shear stress from large-scale blood flow simulations"
Simone Melchionna, Efthimios Kaxiras, Massimo Bernaschi and Sauro S

**1.2 Write a function that reads in the BS object, parses it, converts it into a list of dictionaries...**

In [182]:
content = soup.find('body').text
print(content)
fetched=[]
for i in range(1,90,2):
    body_tag=soup.body
    body_tag
    body_tag.contents
    pub_tag=body_tag.contents[i]





"Approaching the intrinsic band gap in suspended high-mobility graphene nanoribbons"
Ming-Wei Lin, Cheng Ling, Luis A. Agapito, Nicholas Kioussis, Yiyang Zhang, Mark Ming-Cheng Cheng,
PHYSICAL REVIEW B  84,  125411 (2011)






"Effect of symmetry breaking on the optical absorption of semiconductor nanoparticles"
JAdam Gali, Efthimios Kaxiras, Gergely T. Zimanyi, Sheng Meng,
PHYSICAL REVIEW B  84,  035325 (2011)






"Influence of CH2 content and network defects on the elastic properties of organosilicate glasses"
Jan M. Knaup, Han Li, Joost J. Vlassak, and Efthimios Kaxiras,
PHYSICAL REVIEW B  83,  054204 (2011)






"Direct correlation of crystal structure and optical properties in wurtzite/zinc-blende
GaAs nanowire heterostructures"
Martin Heiss, Sonia Conesa-Boj, Jun Ren, Hsiang-Han Tseng, Adam Gali,
PHYSICAL REVIEW B  83,  045303 (2011)






"Endothelial shear stress from large-scale blood flow simulations"
Simone Melchionna, Efthimios Kaxiras, Massimo Bernaschi and Sauro S

**First lets try to find some article links**


In [183]:
#clear output before making pdf
#your code here 
listofurls=[]
for e in soup.select('a'):
    listofurls.append(e['href'])
print(listofurls)

['Papers/2011/PhysRevB_84_125411_2011.pdf', 'Papers/2011/PhysRevB_84_035325_2011.pdf', 'Papers/2011/PhysRevB_83_054204_2011.pdf', 'Papers/2011/PhysRevB_83_045303_2011.pdf', 'Papers/2011/PhilTransRSocA_369_2354_2011.pdf', 'Papers/2011/NewJPhys_13_025025_2011.pdf', 'Papers/2011/NanoLett_11_2962-2967_2011.pdf', 'Papers/2011/Langmuir_27_14248_2011.pdf', 'Papers/2011/JPhysChemLett_2_105-113_2011.pdf', 'Papers/2011/JPhysChemC_115_9276-9282_2011.pdf', 'Papers/2011/JPhysChemC_115_3703-3708_2011.pdf', 'Papers/2011/JChemPhys_134_194706_2011.pdf', 'Papers/2011/Chemistry_17_4496-4506_2011.pdf', 'Papers/2011/CatalSciTechnol_1_1166_2011.pdf', 'Papers/2011/ACSNano_5_1395-1400_2011.pdf', 'Papers/ActaMater_59_44-52_2011.pdf', 'Papers/NewJPhys_12_125012_2010.pdf', 'Papers/PhysRevB_82_201411_2010.pdf', 'Papers/IEEE-SC10_2010.pdf', 'Papers/MolPhys_108_1829-1844_2010.pdf', 'Papers/TopicsCatal_53_365-377_2010.pdf', 'Papers/PhysRevLett_104_155502_2010.pdf', 'Papers/NanoLett_10_1238-1247_2010.pdf', 'Papers/Ph

**OK, now let's try to find the titles...**

In [205]:
all_pubs = soup.select('li')
each_pub = all_pubs[1]
HTML(each_pub.prettify())
each_pub.select_one('a').text[2:][:-1]

'84'

In [185]:
len(all_pubs)

45

In [300]:
def get_pub_title(each_pub):
    return each_pub.select_one('a').text.replace('"', '').replace('\n','')

**Make a list of all the publication titles**



In [301]:
listoftitles = [get_pub_title(each_pub) for each_pub in all_pubs]
print(listoftitles)

['Approaching the intrinsic band gap in suspended high-mobility graphene nanoribbons', 'Effect of symmetry breaking on the optical absorption of semiconductor nanoparticles', 'Influence of CH2 content and network defects on the elastic properties of organosilicate glasses', 'Direct correlation of crystal structure and optical properties in wurtzite/zinc-blendeGaAs nanowire heterostructures', 'Endothelial shear stress from large-scale blood flow simulations', 'Properties of nitrogen-vacancy centers in diamond:the group theoretic approach', 'Lithium-Assisted Plastic Deformation of Silicon Electrodes inLithium-Ion Batteries: A First-Principles Theoretical Study', 'D-π-A Dye System Containing Cyano-Benzoic Acid as AnchoringGroup for Dye-Sensitized Solar Cells', 'New Insights into the Structure of the Vapor/WaterInterface from Large-Scale First-Principles Simulations', 'Design of Dye Acceptors for Photovoltaics from First-PrinciplesCalculations', 'Theoretical Study of O-Assisted Selective C

*hmmm... some of these still need to be cleaned up. The new lines are messing things up. Let's wait and see what the CSV file looks like*

**Next let's try to get the volume number**

In [302]:
def get_pub_vol(each_pub):
    vol_node = each_pub.select_one('b')
    if not vol_node:
        return None
    return each_pub.select_one('b').text

In [482]:
listofvolumes = [get_pub_vol(each_pub) for each_pub in all_pubs]
print(listofvolumes)

['84', '84', '83', '83', '369', '13', '11', '27', '2', '115', '115', '134', '17', '1', '5', '59', '12', '82', None, '108', '53', '104', '10', '81', '6', '181', '22', '150', '103', '2', '180', '113', '25', '79', '06', '79', '113', '102', '102', '79', '102', '130', '113', '15', '78']


In [304]:
len(listofvolumes)

45

**Next let's try to get the journal names**

In [305]:
listofjournals=[]
for e in soup.select('li'):
    listofjournals.append(e('i')[0].text[:-1])
print(listofjournals)

['PHYSICAL REVIEW B', 'PHYSICAL REVIEW B', 'PHYSICAL REVIEW B', 'PHYSICAL REVIEW B', 'Phil. Trans. R. Soc. A', 'New Journal of Physics', 'Nano Lett.', 'Langmuir', 'J. Phys. Chem. Lett.', 'J. Phys. Chem. C', 'J. Phys. Chem. C', 'J. Chem. Phys.', 'Chem. Eur. J.', 'Catal. Sci. Technol.', 'ACSNano.', 'Acta Mater.', 'New J. Phys.', 'Phys. Rev. B', '2010 ACM/IEEE International Conference for High Performance', 'Molec. Phys.', 'Top. Catal.', 'Phys. Rev. Lett.', 'NanoLett.', 'Phys. Rev. B', 'J. Chem. Theory Comput.', 'Comp. Phys. Comm.', 'Concurrency Computat.: Pract. Exper.', 'Sol. St. Comm.', 'Phys. Rev. Lett.', 'Energy & Environmental Sci.', 'Comp. Phys. Comm.', 'J. Phys. Chem. C', 'Int. J. Cardiovasc. Imaging', 'Phys. Rev. B', 'J. Stat. Mech: Th. and Exper.', 'Phys. Rev. E - Rap. Comm.', 'J. Phys. Chem. B', 'Phys. Rev. Lett.', 'Ab initi', 'Phys. Rev. E - Rap. Comm.', 'Phys. Rev. Lett.', 'J. Chem. Phys.', 'J. Phys. Chem. C', 'Sci. Model. Simul.', 'Phys. Rev. B']


In [306]:
len(listofjournals)

45

**Finally let's find the author names**


In [307]:
listofauth=[]
for i in range(1,91,2):
    authors = soup.contents[6].contents[i].contents[1].contents[4]
    listofauth.append(authors[:-4])
print(listofauth)

['Ming-Wei Lin, Cheng Ling, Luis A. Agapito, Nicholas Kioussis, Yiyang Zhang, Mark Ming-Cheng Che', 'JAdam Gali, Efthimios Kaxiras, Gergely T. Zimanyi, Sheng Me', 'Jan M. Knaup, Han Li, Joost J. Vlassak, and Efthimios Kaxir', 'Martin Heiss, Sonia Conesa-Boj, Jun Ren, Hsiang-Han Tseng, Adam Ga', 'Simone Melchionna, Efthimios Kaxiras, Massimo Bernaschi and Sauro Succ', 'J R Maze, A Gali, E Togan, Y Chu, A Trifon', 'Kejie Zhao, Wei L. Wang, John Gregoire, Matt Pharr, Zhigang S', ' Masataka Katono, Takeru Bessho, Sheng Meng, Robin Humphry-Baker, Guido Rothenberg', ' Thomas D. Kuhne, Tod A. Pascal, Efthimios Kaxiras, and Yousung Ju', ' Sheng Meng, Efthimios Kaxiras, Md. K. Nazeeruddin, and Michael Gratz', ' Bingjun Xu, Jan Haubrich, Thomas A. Baker, Efthimios Kaxiras, and Cynthia M. Frie', ' Jun Ren, Sheng Meng, Yi-Lin Wang, Xu-Cun Ma, Qi-Kun Xue, Efthimios Kaxir', ' Jan Haubrich, Efthimios Kaxiras, and Cynthia M. Frie', ' Thomas A. Baker, Bingjun Xu, Stephen C. Jensen, Cynthia M. Friend an

In [308]:
len(listofauth)

45

In [309]:
#ok so now we have a bunch of lists, but we need to compile them into a list of dictionaries...
# So, we have to loop so that we call on the "n_th" element and compile it into the dictionary
# Coded out listoftitles until we can fix the list length
listofdicts=[]
for i in range (0,45):
    td={}
    td['author']=listofauth[i]
    td['title']= listoftitles[i]
    td['URL']=listofurls[i]
    td['journal']=listofjournals[i]
    td['volume']= listofvolumes[i]
    listofdicts.append(td)
print(listofdicts)

[{'author': 'Ming-Wei Lin, Cheng Ling, Luis A. Agapito, Nicholas Kioussis, Yiyang Zhang, Mark Ming-Cheng Che', 'title': 'Approaching the intrinsic band gap in suspended high-mobility graphene nanoribbons', 'URL': 'Papers/2011/PhysRevB_84_125411_2011.pdf', 'journal': 'PHYSICAL REVIEW B', 'volume': '84'}, {'author': 'JAdam Gali, Efthimios Kaxiras, Gergely T. Zimanyi, Sheng Me', 'title': 'Effect of symmetry breaking on the optical absorption of semiconductor nanoparticles', 'URL': 'Papers/2011/PhysRevB_84_035325_2011.pdf', 'journal': 'PHYSICAL REVIEW B', 'volume': '84'}, {'author': 'Jan M. Knaup, Han Li, Joost J. Vlassak, and Efthimios Kaxir', 'title': 'Influence of CH2 content and network defects on the elastic properties of organosilicate glasses', 'URL': 'Papers/2011/PhysRevB_83_054204_2011.pdf', 'journal': 'PHYSICAL REVIEW B', 'volume': '83'}, {'author': 'Martin Heiss, Sonia Conesa-Boj, Jun Ren, Hsiang-Han Tseng, Adam Ga', 'title': 'Direct correlation of crystal structure and optical 

**Let's just pull one out and make sure it looks good**

In [310]:
listofdicts[44]

{'URL': 'Papers/PhysRevB_78_205112_2008.pdf',
 'author': 'E. Manousakis, J. Ren, S. Meng and E. Kaxira',
 'journal': 'Phys. Rev. B',
 'title': 'Effective Hamiltonian for FeAs-based superconductors',
 'volume': '78'}

**1.3 Convert the list of dictionaries into the .bibTex format using python string manipulation (python string formatting on a template string is particularly useful)..**

In [ ]:
# your code here
bibtex_article_template = "@article{{author = \"{author}\"), \n title = \"{title}\"), URL = \"{URL}\"), journal = \"{journal}\"), volume = \"{volume}\")"
publists_bibtex=[]
for i in range (0,45):
    td={}
    td['@article']= listofdicts[i]
    bibtex_article_template.format(**d1)
    publists_bibtex.append(td)
print(publists_bibtex)

*Hmmmmm... the formatting on this is still a bit off. How do we make new lines? We also need to save this to a .bibTex file.*

In [383]:
# your code here

In [ ]:
# check your answer - print the bibTex file
# clear/remove output before making pdf
f = open('publist.bib','r')
print (f.read())

Your output should look like this
```
@article{    
     author = "Ming-Wei Lin, Cheng Ling, Luis A. Agapito, Nicholas Kioussis, Yiyang Zhang, Mark Ming-Cheng Cheng",
     title = "Approaching the intrinsic band gap in suspended high-mobility graphene nanoribbons",
     URL = "Papers/2011/PhysRevB_84_125411_2011.pdf",
     journal = "PHYSICAL REVIEW B",
     volume = 84
}

...

@article{    
     author = "E. Kaxiras and S. Succi",
     title = "Multiscale simulations of complex systems: computation meets reality",
     URL = "Papers/SciModSim_15_59_2008.pdf",
     journal = "Sci. Model. Simul.",
     volume = 15
}

```

** 1.4 Convert the list of dictionaries into the .csv format using pandas, and write the data into `publist.csv`. The csv file should have a header and no integer index...**

In [314]:
# make sure you use head() when printing the dataframe
# your code here
df_pubs_raw = pd.DataFrame(listofdicts)
df_pubs_raw

,URL,author,journal,title,volume
0,Papers/2011/PhysRevB_84_125411_2011.pdf,"Ming-Wei Lin, Cheng Ling, Luis A. Agapito, Nic...",PHYSICAL REVIEW B,Approaching the intrinsic band gap in suspende...,84
1,Papers/2011/PhysRevB_84_035325_2011.pdf,"JAdam Gali, Efthimios Kaxiras, Gergely T. Zima...",PHYSICAL REVIEW B,Effect of symmetry breaking on the optical abs...,84
2,Papers/2011/PhysRevB_83_054204_2011.pdf,"Jan M. Knaup, Han Li, Joost J. Vlassak, and Ef...",PHYSICAL REVIEW B,Influence of CH2 content and network defects o...,83
3,Papers/2011/PhysRevB_83_045303_2011.pdf,"Martin Heiss, Sonia Conesa-Boj, Jun Ren, Hsian...",PHYSICAL REVIEW B,Direct correlation of crystal structure and op...,83
4,Papers/2011/PhilTransRSocA_369_2354_2011.pdf,"Simone Melchionna, Efthimios Kaxiras, Massimo ...",Phil. Trans. R. Soc. A,Endothelial shear stress from large-scale bloo...,369
5,Papers/2011/NewJPhys_13_025025_2011.pdf,"J R Maze, A Gali, E Togan, Y Chu, A Trifon",New Journal of Physics,Properties of nitrogen-vacancy centers in diam...,13
6,Papers/2011/NanoLett_11_2962-2967_2011.pdf,"Kejie Zhao, Wei L. Wang, John Gregoire, Matt P...",Nano Lett.,Lithium-Assisted Plastic Deformation of Silico...,11
7,Papers/2011/Langmuir_27_14248_2011.pdf,"Masataka Katono, Takeru Bessho, Sheng Meng, R...",Langmuir,D-π-A Dye System Containing Cyano-Benzoic Acid...,27
8,Papers/2011/JPhysChemLett_2_105-113_2011.pdf,"Thomas D. Kuhne, Tod A. Pascal, Efthimios Kax...",J. Phys. Chem. Lett.,New Insights into the Structure of the Vapor/W...,2
9,Papers/2011/JPhysChemC_115_9276-9282_2011.pdf,"Sheng Meng, Efthimios Kaxiras, Md. K. Nazeeru...",J. Phys. Chem. C,Design of Dye Acceptors for Photovoltaics from...,115


In [316]:
# your code here
df_pubs_raw.to_csv("data/publist.csv", index=False, header=True)

<hr style='height:2px'>
## Follow the stars in IMDb's list of "The Top 100 Stars for 2017" 

### Overview
In this part, your goal is to extract information from IMDb's Top 100 Stars for 2017 (https://www.imdb.com/list/ls025814950/) and perform some analysis on each star in the list. In particular we are interested to know: a) how many performers made their first movie at 17? b) how many performers started as child actors? c) who is the most proliferate actress or actor in IMDb's list of the Top 100 Stars for 2017? . These questions are addressed in more details in the Questions below. 

When data is not given to us in a file, we need to fetch them using one of the following ways:
- download a file from a source URL
- query a database 
- query a web API 
- scrape data from the web page

<div class='exercise'><b> Question 2 [52 pts]: Web Scraping using Beautiful Soup and exploring using Pandas </b></div>

**2.1** Download the webpage of the "Top 100 Stars for 2017" (https://www.imdb.com/list/ls025814950/) into a `requests` object and name it `my_page`. Explain what the following attributes are:

- `my_page.text`, 
- `my_page.status_code`,
- `my_page.content`.

**2.2** Create a Beautiful Soup object named `star_soup` using `my_page` as input.

**2.3** Write a function called `parse_stars` that accepts `star_soup` as its input and generates a list of dictionaries named `starlist` (see definition below; order of dictionaries does not matter). One of the fields of this dictionary is the `url` of each star's individual page, which you need to scrape and save the contents in the `page` field. Note that there is a ton of information about each star on these webpages.

```
name: the name of the actor/actress as it appears at the top
gender: 0 or 1: translate the word 'actress' into 1 and 'actor' into '0'
url: the url of the link under their name that leads to a page with details
page: BS object with html text acquired by scraping the above 'url' page' 
```

**2.4** Write a function called `create_star_table` which takes `starlist` as an input and extracts information about each star (see function definition for the exact information to be extracted and the exact output definition).  Only extract information from the first box on each star's page. If the first box is acting, consider only acting credits and the star's acting debut, if the first box is Directing, consider only directing credits and directorial debut.

**2.5** Now that you have scraped all the info you need, it's good practice to save the last data structure you created to disk. Save the data structure to a JSON file named `starinfo.json` and submit this JSON file in Canvas. If you do this, if you have to restart, you won't need to redo all the requests and parsings from before.  

**2.6** We provide a JSON file called `data/staff_starinfo.json` created by CS109 teaching staff for consistency, which you should use for the rest of the homework. Import the contents of this JSON file  into a pandas dataframe called `frame`. Check the types of variables in each column and clean these variables if needed. Add a new column to your dataframe with the age of each actor when they made their first appearance, movie or TV, (name this column `age_at_first_movie`). Check some of the values of this new column. Do you find any problems? You don't need to fix them.

**2.7** You are now ready to answer the following intriguing questions: 
- **2.7.1** How many performers made their first appearance (movie or TV) when he/she was 17 years old?

- **2.7.2** How many performers started as child actors? Define child actor as a person younger than 12 years old. 

**2.8** Make a plot of the number of credits against the name of actor/actress. Who is the most prolific actress or actor in IMDb's list of the Top 100 Stars for 2017? Define **most prolific** as the performer with the most credits.
    
### Hints
- Create a variable that groups actors/actresses by the age of their first movie. Use pandas' `.groupby` to divide the dataframe into groups of performers that for example started performing as children (age $<$ 12). The grouped variable is a `GroupBy` pandas object and this object has all of the information needed to then apply operations to each of the groups.
- When cleaning the data make sure the variables with which you are performing calculations are in numerical format.
- The column with the year has some values that are double, e.g. **'2000-2001'** and the column with age has some empty cells. You need to deal with these in a reasonable fashion before performing calculations on the data. 
- You should include both movies and TV shows.
    
### Resources
- The `requests` library makes working with HTTP requests powerful and easy. For more on the `requests` library see http://docs.python-requests.org/

### Answers

In [388]:
import requests

**2.1 Download the webpage of the "Top 100 Stars for 2017 ...**

In [389]:
# your code here
my_page_url = 'https://www.imdb.com/list/ls025814950/'

In [390]:
my_page = requests.get(my_page_url)

- `my_page.text` is ...
- `my_page.status_code` is...
- `my_page.content` is...

**2.2 Create a Beautiful Soup object named star_soup giving my_page as input.**

In [391]:
# your code here
raw_html = my_page.text
print(raw_html[:500])





<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///list/ls025814950?src=mdot">



        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet(


In [392]:
star_soup = BeautifulSoup(raw_html, 'html.parser')

In [428]:
# check your code - you should see a familiar HTML page
# clear/remove output before making pdf
print (star_soup.prettify())

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="app-id=342792525, app-argument=imdb:///list/ls025814950?src=mdot" name="apple-itunes-app"/>
  <script type="text/javascript">
   var IMDbTimer={starttime: new Date().getTime(),pt:'java'};
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <title>
   Top 100 Stars of 2017 - IMDb
  </title>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {w

**2.3 Write a function called `parse_stars` that accepts `star_soup` as its input ...**

```
Function
--------
parse_stars

Input
------
star_soup: the soup object with the scraped page
   
Returns
-------
a list of dictionaries; each dictionary corresponds to a star profile and has the following data:

    name: the name of the actor/actress as it appears at the top
    gender: 0 or 1: translate the word 'actress' into 1 and 'actor' into '0'
    url: the url of the link under their name that leads to a page with details
    page: BS object with 'html text acquired by scraping the above 'url' page' 

Example:
--------
{'name': Tom Hardy,
  'gender': 0,
  'url': https://www.imdb.com/name/nm0362766/?ref_=nmls_hd,
  'page': BS object with 'html text acquired by scraping the 'url' page'
}

```

In [398]:
# parse each actor node
actor_nodes = star_soup.select('.lister-item-content')
len(actor_nodes)

100

In [409]:
actor_node = actor_nodes[3]

In [410]:
HTML(actor_node.prettify())

**This looks good. Let's see if we can just get her name.**

In [411]:
actor_node.select_one('a').text[:-1][1:]

'Alexandra Daddario'

In [415]:
def get_actor_name(actor_node):
    return actor_node.select_one('a').text[:-1][1:]

**Let's get the actor's gender**

In [480]:
actor_node.select_one('p').text

'\n                        Actress |\n Baywatch\n '

In [487]:
def get_actor_gender(actor_node):
    act_gender = actor_node.select_one('p').text
    if "Actress" not in act_gender: 
        return int(0)
    else:
        return int(1)

**Awesome, let's get the URL**

In [450]:
actor_page_url = [get_actor_url(actor_node) for actor_node in actor_nodes]
print (actor_page_url[0])

https://www.imdb.com/name/nm2933757?ref_=nmls_hd


In [442]:
def get_actor_url(actor_node):
    return "https://www.imdb.com" + actor_node.select_one('a')['href']
print("https://www.imdb.com" + actor_node.select_one('a')['href'])

https://www.imdb.com/name/nm1275259?ref_=nmls_hd


**OK let's get the HTML from those individual actor pages**

In [459]:
actor_html = requests.get(actor_page_url[3])

In [461]:
raw_actor_html = actor_html.text
print(raw_actor_html)





<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///name/nm1275259?src=mdot">



        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
  <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
        <title>Alexandra Daddario - IMDb</title>
  <script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "L

**Cool, let's turn it into a beautiful soup object**

In [464]:
actor_soup = BeautifulSoup(raw_actor_html, 'html.parser')

In [465]:
print(actor_soup.prettify())

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="app-id=342792525, app-argument=imdb:///name/nm1275259?src=mdot" name="apple-itunes-app"/>
  <script type="text/javascript">
   var IMDbTimer={starttime: new Date().getTime(),pt:'java'};
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <title>
   Alexandra Daddario - IMDb
  </title>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1}

**OK, let's put it into a function**

In [468]:
def get_actor_page(actor_node):
    actor_page_url = [get_actor_url(actor_node) for actor_node in actor_nodes]
    actor_html = requests.get(actor_page_url)
    raw_actor_html = actor_html.text
    actor_soup = BeautifulSoup(raw_actor_html, 'html.parser')
    return actor_soup
    time.sleep(2)
print(actor_soup)


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///name/nm1275259?src=mdot" name="apple-itunes-app"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Alexandra Daddario - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://www.imdb.

**Let's make a list of actor names**

In [436]:
[get_actor_name(actor_node) for actor_node in actor_nodes]

['Gal Gadot',
 'Tom Hardy',
 'Emilia Clarke',
 'Alexandra Daddario',
 'Bill Skarsgård',
 'Pom Klementieff',
 'Ana de Armas',
 'Dan Stevens',
 'Sofia Boutella',
 'Katherine Langford',
 'Karen Gillan',
 'Margot Robbie',
 'Felicity Jones',
 'Emma Stone',
 'Dylan Minnette',
 'Jennifer Lawrence',
 'Alicia Vikander',
 'Britt Robertson',
 'Ruby Rose',
 'Brie Larson',
 'Keanu Reeves',
 'Sophia Lillis',
 'Jessica Henwick',
 'Cara Delevingne',
 'Haley Bennett',
 'Luke Evans',
 'Teresa Palmer',
 'Tom Holland',
 'Alison Brie',
 'Robin Wright',
 'Zendaya',
 'Emma Watson',
 'Scarlett Johansson',
 'Dafne Keen',
 'Kelly Rohrbach',
 'Eiza González',
 'Laura Haddock',
 'Mary Elizabeth Winstead',
 'Taron Egerton',
 'Anya Taylor-Joy',
 'Elizabeth Debicki',
 'Katheryn Winnick',
 'Sean Young',
 'Bill Paxton',
 'Charlie Hunnam',
 'Yvonne Strahovski',
 'Jason Momoa',
 'Lily James',
 'Jodie Whittaker',
 'Ryan Gosling',
 'Adrianne Palicki',
 'Millie Bobby Brown',
 'Allison Williams',
 'Chris Pratt',
 'Katherine

In [488]:
[get_actor_gender(actor_node) for actor_node in actor_nodes]

[1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0]

**Now a list of URLs**

In [437]:
[get_actor_url(actor_node) for actor_node in actor_nodes]

['https://www.imdb.com/name/nm2933757?ref_=nmls_hd',
 'https://www.imdb.com/name/nm0362766?ref_=nmls_hd',
 'https://www.imdb.com/name/nm3592338?ref_=nmls_hd',
 'https://www.imdb.com/name/nm1275259?ref_=nmls_hd',
 'https://www.imdb.com/name/nm0803889?ref_=nmls_hd',
 'https://www.imdb.com/name/nm2962353?ref_=nmls_hd',
 'https://www.imdb.com/name/nm1869101?ref_=nmls_hd',
 'https://www.imdb.com/name/nm1405398?ref_=nmls_hd',
 'https://www.imdb.com/name/nm1154749?ref_=nmls_hd',
 'https://www.imdb.com/name/nm7692698?ref_=nmls_hd',
 'https://www.imdb.com/name/nm2394794?ref_=nmls_hd',
 'https://www.imdb.com/name/nm3053338?ref_=nmls_hd',
 'https://www.imdb.com/name/nm0428065?ref_=nmls_hd',
 'https://www.imdb.com/name/nm1297015?ref_=nmls_hd',
 'https://www.imdb.com/name/nm1910255?ref_=nmls_hd',
 'https://www.imdb.com/name/nm2225369?ref_=nmls_hd',
 'https://www.imdb.com/name/nm2539953?ref_=nmls_hd',
 'https://www.imdb.com/name/nm1429380?ref_=nmls_hd',
 'https://www.imdb.com/name/nm3199307?ref_=nml

**Now a list of soup objects**

**Ok let's put this all into a list of dictionaries**

In [19]:
# your code here 
def parse_stars(star_soup):
    

This should give you 100

In [20]:
len(starlist)

In [21]:
# check your code
# this list is large because of the html code into the `page` field
# to get a better picture, print only the first element
# clear/remove output before making pdf
starlist[1]

Your output should look like this:
```
{'name': 'Gal Gadot',
 'gender': 1,
 'url': 'https://www.imdb.com/name/nm2933757?ref_=nmls_hd',
 'page': 
 <!DOCTYPE html>
 
 <html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
 <meta charset="utf-8"/>
 <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
 <meta content="app-id=342792525, app-argument=imdb:///name/nm2933757?src=mdot" name="apple-itunes-app"/>
 <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
 <script>
     if (typeof uet == 'function') {
       uet("bb", "LoadTitle", {wb: 1});
     }
 </script>
 <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
 
... 


 ```

**2.4 Write a function called `create_star_table` to extract information about each star ...**

```
Function
--------
create_star_table

Input
------
the starlist
   
Returns
-------

a list of dictionaries; each dictionary corresponds to a star profile and has the following data:

    star_name: the name of the actor/actress as it appears at the top
    gender: 0 or 1 (1 for 'actress' and 0 for 'actor')  
    year_born : year they were born
    first_movie: title of their first movie or TV show
    year_first_movie: the year they made their first movie or TV show
    credits: number of movies or TV shows they have made in their career.
    
--------
Example:

{'star_name': Tom Hardy,
  'gender': 0,
  'year_born': 1997,
  'first_movie' : 'Batman',
  'year_first_movie' : 2017,
  'credits' : 24}
  
```

In [22]:
def create_star_table(starlist: list) -> list:
# your code here


In [23]:
# check your code
# clear/remove output before making the pdf file
star_table

Your output should look like this (the order of elements is not important):
```
[{'name': 'Gal Gadot',
  'gender': 1,
  'year_born': '1985',
  'first_movie': 'Bubot',
  'year_first_movie': '2007',
  'credits': '25'},
 {'name': 'Tom Hardy',
  'gender': 0,
  'year_born': '1977',
  'first_movie': 'Tommaso',
  'year_first_movie': '2001',
  'credits': '55'},
  
...
```

**2.5 Now that you have scraped all the info you need, it's a good practice to save the last data structure you ...**

In [24]:
# your code here


To check your JSON saving, re-open the JSON file and reload the code

In [25]:
with open("starinfo.json", "r") as fd:
    star_table = json.load(fd)
    
# output should be the same
# clear/remove output before making the pdf file
star_table

**2.6 Import the contents of the staff's JSON file (`data/staff_starinfo.json`) into a pandas dataframe. ...**

In [26]:
# your code here


In [27]:
# your code here 


In [28]:
# your code here


In [31]:
# your code here


*your answer here*


**2.7 You are now ready to answer the following intriguing questions**:

**2.7.1 How many performers made their first movie at 17?**

In [32]:
# your code here


Your output should look like this:<BR>
8 performers made their first movie at 17

**2.7.2 How many performers started as child actors? Define child actor as a person less than 12 years old.** 

In [33]:
# your code here


**2.8 Make a plot of the number of credits versus the name of actor/actress.**

In [34]:
# your code here


In [35]:
# your code here 


<hr style="height:2px">
##  Going the Extra Mile 
Be sure to complete problems 1 and 2 before tackling this problem...it is worth only 8 points.


<div class='exercise'> <b>Question 3 [8 pts]: Parsing using Regular Expressions (regex) </b></div>

Even though scraping HTML with regex is sometimes considered bad practice, you are to use python's **regular expressions** to answer this problem.  Regular expressions are useful to parse strings, text, tweets, etc. in general (for example, you may encounter a non-standard format for dates at some point). Do not use BeautifulSoup to answer this problem.

 **3.1** Write a function called `get_pubs` that takes an .html filename as an input and returns a string containing the HTML page in this file (see definition below). Call this function using `data/publist_super_clean.html` as input and name the returned string `prof_pubs`. 
 
 **3.2** Calculate how many times the author named '`C.M. Friend`' appears in the list of publications. 
 
 **3.3** Find all unique journals and copy them in a variable named `journals`.  
 
 **3.4** Create a list named `pub_authors` whose elements are strings containing the authors' names for each paper. 
    
### Hints
- Look for patterns in the HTML tags that reveal where each piece of information such as the title of the paper, the names of the authors, the journal name, is stored. For example, you might notice that the journal name(s) is contained between the &lt;I&gt; HTML tag. 
- Learning about your domain is always a good idea: you want to check the names to make sure that they belong to actual journals. Thus, while journal name(s) is contained between the &lt;I&gt; HTML tag, please note that <i>all</i> strings found between &lt;I&gt; tags may not be journal names.
- Each publication has multiple authors. 
- `C.M. Friend` also shows up as `Cynthia M. Friend` in the file.  Count just `C. M. Friend`. 
- There is a comma at the end of the string of authors. You can choose to keep it in the string or remove it and put it back when you write the string as a BibTex entry. 
- You want to remove duplicates from the list of journals. Duplicates may also occur due to misspellings or spaces, such as: `Nano Lett.`, and `NanoLett.` You can assume that any journals with the same initials (e.g., `NL` for `NanoLett.`) are the same journal.

### Resources
- **Regular expressions:** a) https://docs.python.org/3.3/library/re.html, b) https://regexone.com, and c) https://docs.python.org/3/howto/regex.html. 
- ** HTML:** if you are not familiar with HTML see https://www.w3schools.com/html/ or one of the many tutorials on the internet. 
- ** Document Object Model (DOM):** for more on this programming interface for HTML and XML documents see https://www.w3schools.com/js/js_htmldom.asp. 

### Answers

** 3.1  Write a function called `get_pubs` that takes an .html filename as an input and returns a string  ... **    

In [36]:
# first import the necessary reg expr library
import re

In [37]:
# use this file provided
PUB_FILENAME = 'data/publist_super_clean.html'

In [38]:
# your code here


In [39]:
# your code here


In [40]:
# checking your code 
# clear/remove output before creating the pdf file
print(prof_pubs)

You should see an HTML page that looks like this (colors are not important)
```html
<LI>
<A HREF="Papers/2011/PhysRevB_84_125411_2011.pdf" target="paper244">
&quot;Approaching the intrinsic band gap in suspended high-mobility graphene nanoribbons&quot;</A>
<BR>Ming-Wei Lin, Cheng Ling, Luis A. Agapito, Nicholas Kioussis, Yiyang Zhang, Mark Ming-Cheng Cheng,
<I>PHYSICAL REVIEW B </I> <b>84</b>,  125411 (2011)
<BR>
</LI>
</OL>

<OL START=243>
<LI>
<A HREF="Papers/2011/PhysRevB_84_035325_2011.pdf" target="paper243">
&quot;Effect of symmetry breaking on the optical absorption of semiconductor nanoparticles&quot;</A>
<BR>JAdam Gali, Efthimios Kaxiras, Gergely T. Zimanyi, Sheng Meng,
<I>PHYSICAL REVIEW B </I> <b>84</b>,  035325 (2011)
<BR>
</LI>
</OL>

<OL START=242>
<LI>
<A HREF="Papers/2011/PhysRevB_83_054204_2011.pdf" target="paper242">
&quot;Influence of CH2 content and network defects on the elastic properties of organosilicate glasses&quot;</A>
<BR>Jan M. Knaup, Han Li, Joost J. Vlassak, and Efthimios Kaxiras,
<I>PHYSICAL REVIEW B </I> <b>83</b>,  054204 (2011)
<BR>
</LI>
</OL>
    ```

**3.2 Calculate how many times the author ... **

In [41]:
# your code here


**3.3 Find all unique journals and copy ...**

In [42]:
# your code here 


In [43]:
# check your code 
journals

Your output should look like this (no duplicates):
```
{'2010 ACM/IEEE International Conference for High Performance',
 'ACSNano.',
 'Ab initio',
 'Acta Mater.',
 'Catal. Sci. Technol.',
 'Chem. Eur. J.',
 'Comp. Phys. Comm.',
 'Concurrency Computat.: Pract. Exper.',
 'Energy & Environmental Sci.',
 'Int. J. Cardiovasc. Imaging',
 'J. Chem. Phys.',
 'J. Chem. Theory Comput.',
 'J. Phys. Chem. B',
 'J. Phys. Chem. C',
 'J. Phys. Chem. Lett.',
 'J. Stat. Mech: Th. and Exper.',
 'Langmuir',
 'Molec. Phys.',
 'Nano Lett.',
 'New Journal of Physics',
 'PHYSICAL REVIEW B',
 'Phil. Trans. R. Soc. A',
 'Phys. Rev. E - Rap. Comm.',
 'Phys. Rev. Lett.',
 'Sci. Model. Simul.',
 'Sol. St. Comm.',
 'Top. Catal.'}
 ```

**3.4 Create a list named `pub_authors`...**

In [44]:
# your code here


In [45]:
# check your code: print the list of strings containing the author(s)' names
for item in pub_authors:
    print (item)

Your output should look like this (a line for each paper's authors string of names)
```
Ming-Wei Lin, Cheng Ling, Luis A. Agapito, Nicholas Kioussis, Yiyang Zhang, Mark Ming-Cheng Cheng,
JAdam Gali, Efthimios Kaxiras, Gergely T. Zimanyi, Sheng Meng,
Jan M. Knaup, Han Li, Joost J. Vlassak, and Efthimios Kaxiras,
Martin Heiss, Sonia Conesa-Boj, Jun Ren, Hsiang-Han Tseng, Adam Gali,

...

T.A. Baker, C.M. Friend and E. Kaxiras,
T.A. Baker, C.M. Friend and E. Kaxiras,
E. Kaxiras and S. Succi,
E. Manousakis, J. Ren, S. Meng and E. Kaxiras,
```